For general steps, just read the text in between the code.
For more detailed steps, read the comments in the code.

This program will take the data from level 1 to level 2. In essence that means from slightly cleaned up data to objectified data which is also sliced into many different sets.

First an initalization is done by importing many useful libraries, after which the data pickles from the previous program are imported.

In [1]:
import numpy as np                 #number crunching library
import pandas as pd                #Numpy on big data steroids
from datetime import datetime      #does all the heavy lifting with dates and times
from datetime import date
import dateutil
from random import sample          #(psuedo)randomness
import random 


pd.set_option("display.precision", 2)     #makes pandas not output too many decimals

df_11_SE = pd.read_pickle('data/[11SE].pkl')    #imports the single early flight dataset in level 1
df_11 = pd.read_pickle('data/[11].pkl')         #imports all flights in level 1

df_11_SE.head() #shows top of dataset


,FLIGHT_ID,TRACK_ID,X,Y,MODE_C,CALLSIGN,ICAO_ACTYPE,DEST,ADEP,FLIGHT_TYPE,RADAR,LANDING_TIME,TIME,TAKEOFF_TIME
0,5170097,135322667,9586,18498,400.0,KLM7451,B737,EHAM,ENGM,INBOUND,ARTACC,22-10-2010 0:51:33,22-10-2010 0:20:01,NaN
1,5170097,135322667,9562,18432,400.0,KLM7451,B737,EHAM,ENGM,INBOUND,ARTACC,22-10-2010 0:51:33,22-10-2010 0:20:06,NaN
2,5170097,135322667,9536,18362,400.0,KLM7451,B737,EHAM,ENGM,INBOUND,ARTACC,22-10-2010 0:51:33,22-10-2010 0:20:10,NaN
3,5170097,135322667,9512,18286,400.0,KLM7451,B737,EHAM,ENGM,INBOUND,ARTACC,22-10-2010 0:51:33,22-10-2010 0:20:15,NaN
4,5170097,135322667,9486,18216,400.0,KLM7451,B737,EHAM,ENGM,INBOUND,ARTACC,22-10-2010 0:51:33,22-10-2010 0:20:20,NaN


We define a flight as an object now. Therefore we can easily apply certain functions to it. This has already been done in a seperate file, named flightObj.py, so that this object can be used everywhere, anywhere.

In [2]:
# class Flight():    #main class for flights
#     def __init__(self, flightid, data):         #starts object
#         self.fid = flightid                     #using flight ID
#         self.df = data
#         self.df.reset_index(drop=True)      #solves issues where the index stays from previous, resets it to start at 1,2,3.. etc
        
#         self.data_prepare()                 #main pointer to data prepare function
        
#     def data_prepare(self):                 #prepare function, seperate so it can be called seperately
        
#         self.populate()
#         self.make_dataset()
        
        
        
#     def populate(self):                    #take constant values from the data and attach it to the object
#         self.tid = self.df['TRACK_ID'].iloc[0]
#         self.call = self.df['CALLSIGN'].iloc[0]
#         self.actype = self.df['ICAO_ACTYPE'].iloc[0]
#         self.dest = self.df['DEST'].iloc[0]
#         self.adep = self.df['ADEP'].iloc[0]
#         self.ftype = self.df['FLIGHT_TYPE'].iloc[0]
#         self.radar = self.df['RADAR'].iloc[0]
#         if self.ftype == 'OUTBOUND':                         #the data has different collumns for inbound and outbound flights
#             self.aptime = self.df['TAKEOFF_TIME'].iloc[0]
#         else:
#             self.aptime = self.df['LANDING_TIME'].iloc[0]
    
#     def get_tplus(self, start, row):                         #a function that gives the difference in seconds between given row of data and the start time of the flight.
#         tplus = datetime.strptime(row['TIME'], '%d-%m-%Y %H:%M:%S') - start
#         return tplus
    
#     def make_dataset(self):
#         self.ds = self.df[['X', 'Y', 'MODE_C', 'TIME']]
#         self.first_timestamp = datetime.strptime(self.ds['TIME'].iloc[0], '%d-%m-%Y %H:%M:%S')
#         #self.first_timestamp = datetime.strptime('2019', '%Y')
#         self.ds['TPLUS'] = self.ds.apply(lambda row : self.get_tplus(self.first_timestamp, row), axis=1).apply(lambda x: int(x.total_seconds()))

#     #This part calculates the velocity by substracting taking the derivative with respect to time before and after a specific point. [a, b, c, d]: (b-a) - (b-c) = c-a
#         ds_c = self.ds[['X', 'Y', 'MODE_C', 'TPLUS']].diff(periods = 1) - self.ds[['X', 'Y', 'MODE_C', 'TPLUS']].diff(periods = -1)
#         self.ds['VEL_X'] = ds_c['X'] / ds_c['TPLUS']
#         self.ds['VEL_Y'] = ds_c['Y'] / ds_c['TPLUS']
#         self.ds['VEL_Z'] = ds_c['MODE_C'] / ds_c['TPLUS']
        
#         self.ds['VEL_TOT'] = np.sqrt(self.ds['VEL_X']**2 + self.ds['VEL_Y']**2 + self.ds['VEL_Z']**2)

from flightObj import Flight        #moved this class to a seperate file so it can easily be used everywhere. This version is therefore outdated, see flightObj.py
        
        
        

We want to check how many flight we have in our data. This is done by using the following:

In [3]:
flightgroups = df_11.groupby('FLIGHT_ID')   #the groupby function picks apart a Pandas dataframe by the value you specify, in this case the unique flight id
flights = []

for name, value in flightgroups:  #the groupby produces a dictionary, this gets iterated through
    #print(type(name))
    data = pd.DataFrame(flightgroups.get_group(name))     #getting the data back from a groupby is not the easiest, it was not meant for this operation. Here a dataframe is made per flight using the groupby function get_group
    #print(type(data))
    
    flight = Flight(str(name), data)      #objectify!
    flights.append(flight)                #add to simple list of all flights

    
print('Amount of flights: ', len(flights))

/home/user/CD11Research/flightObj.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.ds['TPLUS'] = self.ds.apply(lambda row : self.calc_tplus(self.first_timestamp, row), axis=1).apply(lambda x: int(x.total_seconds()))
/home/user/CD11Research/flightObj.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.ds['VEL_X'] = ds_c['X'] / ds_c['TPLUS']
/home/user/CD11Research/flightObj.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

Amount of flights:  919


Now we want to know some basic information about the flight, like type of aircraft, location of department/arrival.

In [4]:
for index, flight in zip(range(20),flights):    #for the first x number of flights, print out some basic info
    print('______')
    print('Callsign ', flight.call)
    print('Aircraft ', flight.actype)
    print('From     ', flight.adep)
    print('To       ', flight.dest)
    if flight.ftype == 'OUTBOUND':
        print('Took off at ', flight.aptime)
    else:
        print('Landing at ', flight.aptime)
    print('\n')

______
Callsign  KLM7451
Aircraft  B737
From      ENGM
To        EHAM
Landing at  22-10-2010 0:51:33


______
Callsign  TRA698
Aircraft  B738
From      GCFV
To        EHAM
Landing at  22-10-2010 0:37:45


______
Callsign  TRA458
Aircraft  B738
From      HEGN
To        EHAM
Landing at  22-10-2010 1:21:24


______
Callsign  TFL480
Aircraft  B738
From      HEGN
To        EHAM
Landing at  22-10-2010 1:10:24


______
Callsign  CAI514
Aircraft  B734
From      EHAM
To        LTFE
Took off at  22-10-2010 1:40:37


______
Callsign  BCS6367
Aircraft  B752
From      EDDP
To        EHAM
Landing at  22-10-2010 2:19:03


______
Callsign  CAI804
Aircraft  B734
From      EHAM
To        LTAI
Took off at  22-10-2010 2:06:27


______
Callsign  KLM754
Aircraft  MD11
From      TNCB
To        EHAM
Landing at  22-10-2010 2:31:51


______
Callsign  UAE9990
Aircraft  B744
From      EHAM
To        OMDB
Took off at  22-10-2010 2:33:06


______
Callsign  KLM890
Aircraft  B744
From      VHHH
To        EHAM
Landing

Now what we want to do, is slice the data into some smaller data samples of around 20 flights in a certain categories(see appendix A.3 in research plan) which we can use in our furher calculations.

In [5]:
random.seed(31415)   #random seeds make sure that randomness is only random once. When this section is re-run the exact same output will be gotten as the first random time, which is crucial.

se_21 = pd.Series(flights) #all flights in a series

se_21_O = pd.Series(sample([flight for flight in flights if flight.ftype == 'OUTBOUND'], 20))  #random sample of 20 outbound flights

se_21_I = pd.Series(sample([flight for flight in flights if flight.ftype == 'INBOUND'], 20))   #random sample of 20 inbound flights

se_21_E = pd.Series(sample([flight for flight in flights if flight.first_timestamp < dateutil.parser.isoparse('2010-10-22 08:00:00')],20))  #random sample of 20 early (before 8h in the morning) flights

se_21_M = pd.Series(sample([flight for flight in flights if (flight.first_timestamp > dateutil.parser.isoparse('2010-10-22 08:00:00') and flight.first_timestamp < dateutil.parser.isoparse('2010-10-22 16:00:00')) ],20))
                                                                                                                                            #random sample of 20 midday (between 8h and 16h) flights

se_21_L = pd.Series(sample([flight for flight in flights if (flight.first_timestamp > dateutil.parser.isoparse('2010-10-22 16:00:00') and flight.first_timestamp < dateutil.parser.isoparse('2010-10-22 23:59:59'))],20))
                                                                                                                                            #random sample of 20 late (after 16h but still in the same day) flights
    
se_21_S = pd.Series(sample(flights,1))

se_21_T_A320 = pd.Series(sample([flight for flight in flights if flight.actype == 'A320'], 20))

for flight in se_21_L:  #just a check if the flights are actually what they are supposed to
    print(flight.call, '  \t', flight.actype, ' \t', flight.first_timestamp)

EIN609   	 A320  	 2010-10-22 16:08:29
KLM1749   	 F70  	 2010-10-22 19:15:07
KLM1273   	 E190  	 2010-10-22 18:16:24
ISS3423   	 A319  	 2010-10-22 16:07:24
KLM35E   	 B737  	 2010-10-22 19:07:31
ISS3424   	 A319  	 2010-10-22 17:37:31
RAM851G   	 B735  	 2010-10-22 17:04:48
EZY7012   	 A319  	 2010-10-22 20:36:38
KLM1799   	 E190  	 2010-10-22 17:06:09
KLM56T   	 F70  	 2010-10-22 17:29:40
CFE92A   	 E170  	 2010-10-22 16:30:10
KLM1270   	 F70  	 2010-10-22 21:06:08
KLM71G   	 F70  	 2010-10-22 19:16:38
KLM1045   	 F70  	 2010-10-22 18:53:55
KLM78C   	 F70  	 2010-10-22 16:43:17
SAS554   	 CRJ9  	 2010-10-22 18:27:07
KLM1732   	 E190  	 2010-10-22 16:40:41
KLM1190   	 B737  	 2010-10-22 16:12:45
KLM50B   	 B734  	 2010-10-22 16:54:53
KLM1480   	 F70  	 2010-10-22 18:30:44


Finally, we need to put these data samples into pickles again, to easily switch between programs later

In [6]:
#se_21.to_pickle('data/[21].pkl')  #putting everything in python pickles again

#se_21_O.to_pickle('data/[21_O].pkl')
se_21_I.to_pickle('data/[21_I].pkl')
#se_21_E.to_pickle('data/[21_E].pkl')
#se_21_M.to_pickle('data/[21_M].pkl')
se_21_L.to_pickle('data/[21_L].pkl')


import pickle
with open('data/[21_O].pkl', 'wb') as f:
     pickle.dump(se_21_O, f)
with open('data/[21_I].pkl', 'wb') as f:
     pickle.dump(se_21_I, f)
with open('data/[21_E].pkl', 'wb') as f:
     pickle.dump(se_21_E, f)
with open('data/[21_M].pkl', 'wb') as f:
     pickle.dump(se_21_M, f)
with open('data/[21_L].pkl', 'wb') as f:
     pickle.dump(se_21_L, f)
with open('data/[21_S].pkl', 'wb') as f:
     pickle.dump(se_21_S, f)
with open('data/[21_T_A320].pkl', 'wb') as f:
     pickle.dump(se_21_T_A320, f)